In [1]:
# Importing libraries
import sys
import numpy as np
import pandas as pd
import cantera as ct
import scipy.optimize as opt

# Importing packages
import gasifier as g
import fuel as fu
import energy as en
import outputs as op
import feedstock as fs
import pp

For species Fe(c), discontinuity in cp/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 4.2881898

For species Fe(c), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 3.425820683333331

For species Fe(c), discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.0
	Value computed using high-temperature polynomial: 8.486560101557188

For species Cr(cr), discontinuity in cp/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  749.1520512400002
	Value computed using high-temperature polynomial: 0.0

For species Cr(cr), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  113.24055562733336
	Value computed using high-temperature polynomial: 0.0

For species Cr(cr), discontinuity in s/R detec

In [2]:
# # Old packages
# import feedstock as fs1
# import pp as pp1
# import gasifier as g1

In [3]:
fuelID = "AltafiniSawdust"
Info = {"Description":"Pinus Elliotis sawdust", "Type":"Biomass", "Category":"Wood", "Reference":"Altafini et al.", "Year":2003, "DOI":"10.1016/S0196-8904(03)00025-6"}
moist = [None, None]
prox = ["db", 0.0, 99.9, 0.1]
HV = [20.407, 19.087]
biochem = [None, None, None]
ult = ["db", 52.00, 41.55, 0.28, 6.07, 0.00, 0.00]
ashC = [None, None, None, None, None, None, None, None, None, None, None]

#fu.addToDatabase(fuelID, Info, moist, prox, HV, biochem, ult, ashC)

In [4]:
print(pp.nsp)

90


In [5]:
fuelID = "AltafiniSawdust"
species = ['H2', 'CO', 'CH4', 'CO2', 'N2']
experimental = np.array([20.59, 26.73, 0.02, 7.28, 44.85])/100 # fraction
T0 = 800+273.15 # K
P = ct.one_atm # Pa
air = 2.0841 # kg/kg fuel
airType = 'air'
moist = 0

In [6]:
fuelID = "HighAshCoal"
species = ['H2', 'CO', 'CH4', 'CO2', 'H2O', 'H2S', 'NH3', 'N2']
#experimental = np.array([10.2, 15.7, 8.0, 1.0, 65.1])/100 # fraction
T0 = 1000+273.15 # K
P = ct.one_atm # Pa
air = 0.4
airType = 'ER'
moist = 0

In [7]:
fuelID = "Highvale"
species = ['CO', 'CO2', 'H2', 'CH4', 'N2']
experimental = np.array([9.1, 15.0, 5.6, 0.5, 69.8])/100 # fraction
CCexp = 73.8 # %
T0 = 880+273.15 # K
P = 1.55E5 # Pa
air = 0.52
airType = 'ER'
moist = 0.11607

In [8]:
fuelID = "Highvale"
species = ['CO', 'CO2', 'H2', 'CH4', 'N2']
experimental = np.array([10.2, 15.7, 8.0, 1.0, 65.1])/100 # fraction
CCexp = 61.4 # %
T0 = 810+273.15 # K
P = 1.6E5 # Pa
air = 0.37
airType = 'ER'
moist = 0.11607

In [9]:
report0 = g.gasifier(fuelID, moist=moist, T=T0, P=P, air=air, airType=airType, species=species+['C'], C_avail=1.0)
CC0 = report0['CC']
C_avail0 = CCexp / CC0
print(C_avail0)

0.6227745197925871


In [10]:
DT1, C_avail1, sqerr = g.findParams(fuelID, experimental, moist=moist, T0=T0, P=P, air=air, airType=airType, species=species)
print(DT1)
print(C_avail1)
print(sqerr)
print(sqerr*100, '%')

0.02417773437499986
0.5144531249999789
0.0661669735330218
6.61669735330218 %


In [11]:
Tquasi = g.findTquasi(fuelID, experimental, moist=moist, T0=T0, P=P, 
                air=air, airType=airType, C_avail=C_avail0, species=species)
DT2 = T0 - Tquasi.x[0]
print(DT2)
print(Tquasi)

279.563168127537
 final_simplex: (array([[803.58683187],
       [803.58688352]]), array([0.01320056, 0.01320056]))
           fun: 0.013200558304511827
       message: 'Optimization terminated successfully.'
          nfev: 50
           nit: 25
        status: 0
       success: True
             x: array([803.58683187])


In [12]:
report1 = g.gasifier(fuelID, moist=moist, T=T0-DT1, air=air, P=P, airType=airType, species=species+['C'], C_avail=C_avail1)
CC1 = report1['CC']
print(CC1)
for sp in species+['C']:
    print(sp, report1[sp]*100, '%')

51.15670581983078
CO 21.615255228987102 %
CO2 7.264273724438117 %
H2 18.326220255374555 %
CH4 0.02655097776255576 %
N2 46.32177383697563 %
C 3.010811893063498e-26 %


In [13]:
report2 = g.gasifier(fuelID, moist=moist, T=T0-DT2, air=air, P=P, airType=airType, species=species+['C'], C_avail=C_avail0)
CC2 = report1['CC']
print(CC2)
for sp in species+['C']:
    print(sp, report2[sp]*100, '%')

51.15670581983078
CO 4.045425207567627 %
CO2 16.910218943581462 %
H2 9.891175392446197 %
CH4 2.633638271324082 %
N2 55.80393395507851 %
C 2.8778485436630257e-37 %


In [14]:
def ARE(predicted, experimental):
    ARE = 0
    for i, exp in enumerate(experimental):
        ARE += abs((predicted[i]-exp)/exp)
    return ARE/len(predicted)

def AAE(predicted, experimental):
    AAE = 0
    for i, exp in enumerate(experimental):
        print(predicted[i], exp)
        AAE += abs((predicted[i]-exp))
    return AAE/len(predicted)

def SQE(predicted, experimental):
    SQE = 0
    for i, exp in enumerate(experimental):
        SQE += (predicted[i]-exp)**2
    return SQE

predicted = []
for i, sp in enumerate(species):
    predicted.append(report2[sp])

error = AAE(predicted, experimental)
print(error*100, '%')

0.04045425207567627 0.102
0.16910218943581462 0.157
0.09891175392446197 0.08
0.02633638271324082 0.01
0.5580393395507851 0.6509999999999999
4.037134688941118 %
